# BAX pore assembly
Initial condition:
\begin{equation*}
P_0(x)=\gamma \cdot \exp\left(-\frac{1}{2} (x-\mu)^T C^{-1} (x-\mu)\right),
\end{equation*}
with $C=0.2$, $\mu=(40,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,50,\,0)$ and $\gamma$ was determined by the condition $\sum_{x\in\Omega^{\zeta,\eta}} P_0(x)=1$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scripts.models.bax_pysb as pysb_model
from pysb.simulator import StochKitSimulator

from ssa_helper import *

## Simulation parameters

In [ ]:
n_time = 1 + 1
tspan = np.linspace(0.0, 145, n_time)

# This defines the small sample space
liml = np.array([38, 0, 0, 0, 0, 0, 0, 0, 0, 48, 0])
n = np.array([5, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4])

sweeps = 1e4
meta_runs = 1

m = n.size # len(pysb_model.model.observables)
observables = [obs.name for obs in pysb_model.model.observables]

## Set up the initial condition

In [ ]:
mu = np.array([40.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 50.0, 0.0])
C = 0.2
Cinv = 1 / C

# Calculate the normalization
sum_vec = np.zeros(n.size)
for i, n_el in enumerate(n):
    for x in range(n_el):
        x_vec = np.array([x])
        sum_vec[i] += np.exp(-0.5 * Cinv * np.linalg.norm(x + liml[i] - mu[i]) ** 2)
gamma = 1 / np.prod(sum_vec)

@njit
def eval_P0(x: np.ndarray) -> float:
    return gamma * np.exp(-0.5 * Cinv * np.linalg.norm(x - mu) ** 2)

## Run StochKit simulation

In [ ]:
n_runs, n_runs_tot = calculateNRuns(eval_P0, sweeps, n, liml)
result_tot = np.empty((meta_runs * n_runs_tot, n_time, m), dtype="int64")

for i in range(meta_runs):
    result_tot[i * n_runs_tot : (i + 1) * n_runs_tot] = runSSA(n_runs, n_runs_tot, tspan, n, liml, pysb_model.model, observables)
    print(i)

np.save("scripts/reference_solutions/bax_ssa_general_ref_1e4.npy", result_tot)